In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np 
import matplotlib.pyplot as plt 


In [2]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


### Define FOURSQUARE Credentials

In [3]:
CLIENT_ID = '4IZV4ELEAZEQOKLI1BUYGLCNHQLVXZ2OR33RY5RDR4NO4ZXB' # your Foursquare ID
CLIENT_SECRET = 'FOKYDWC0Q1XJ1ZRK2LO4YYAN1YCA4NO05R1ZCFP24EXHDQIJ' # your Foursquare Secret
VERSION = '20190604'
LIMIT = 30
#print('Your credentails:')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

### A tourist who is staying in a Mariott City Centre Hotel wants to explore the area within 1.5 kms

In [4]:
address = 'One Blue Jays Way, Toronto, ON'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6444703 -79.3916619


In [13]:
radius = 1500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)

In [6]:
import requests
results = requests.get(url).json()
print ('There are {} venues around Mariott City Centre Hotel.'.format(len(results['response']['groups'][0]['items'])))
results

There are 30 venues around Mariott City Centre Hotel.


{'meta': {'code': 200, 'requestId': '5d5ede9fe0c0c90039e373ca'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Entertainment District',
  'headerFullLocation': 'Entertainment District, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 239,
  'suggestedBounds': {'ne': {'lat': 43.657970313500016,
    'lng': -79.37304093571359},
   'sw': {'lat': 43.63097028649999, 'lng': -79.41028286428642}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c05cf964a520c9f520e3',
       'name': 'The Second City',
       'location': {'address': '51 Mercer St',
        'crossStreet': 'at Blue Jays Way',
        'lat': 43.64563251724917,
        'lng': -79.39122468317156,
  

In [10]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4ad4c05cf964a520c9f520e3',
  'name': 'The Second City',
  'location': {'address': '51 Mercer St',
   'crossStreet': 'at Blue Jays Way',
   'lat': 43.64563251724917,
   'lng': -79.39122468317156,
   'labeledLatLngs': [{'label': 'display',
     'lat': 43.64563251724917,
     'lng': -79.39122468317156}],
   'distance': 134,
   'postalCode': 'M5V 9G9',
   'cc': 'CA',
   'city': 'Toronto',
   'state': 'ON',
   'country': 'Canada',
   'formattedAddress': ['51 Mercer St (at Blue Jays Way)',
    'Toronto ON M5V 9G9',
    'Canada']},
  'categories': [{'id': '4bf58dd8d48988d18e941735',
    'name': 'Comedy Club',
    'pluralName': 'Comedy Clubs',
    'shortName': 'Comedy Club',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/arts_entertainment/comedyclub_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count

### Define a function to retrieve the venue type and insert into a data frame

In [7]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']



In [11]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,The Second City,Comedy Club,51 Mercer St,CA,Toronto,Canada,at Blue Jays Way,134,"[51 Mercer St (at Blue Jays Way), Toronto ON M...","[{'label': 'display', 'lat': 43.64563251724917...",43.645633,-79.391225,NaN,M5V 9G9,ON,4ad4c05cf964a520c9f520e3
1,TIFF Bell Lightbox,Movie Theater,350 King St. W,CA,Toronto,Canada,at John St.,244,"[350 King St. W (at John St.), Toronto ON M5V ...","[{'label': 'display', 'lat': 43.64634159071232...",43.646342,-79.390063,Entertainment District,M5V 3X5,ON,4bcf714ab221c9b67f0ad2d0
2,Bar Hop,Beer Bar,391 King St W,CA,Toronto,Canada,at Charlotte St,199,"[391 King St W (at Charlotte St), Toronto ON M...","[{'label': 'display', 'lat': 43.64560340159375...",43.645603,-79.393583,NaN,M5V 1K1,ON,4fcc0dd177167574f0de38d5
3,SOMA chocolatemaker,Dessert Shop,443 King St. W,CA,Toronto,Canada,NaN,340,"[443 King St. W, Toronto ON M5V 1K4, Canada]","[{'label': 'display', 'lat': 43.64532829235515...",43.645328,-79.395714,NaN,M5V 1K4,ON,4df796f1aeb7da11e1a974d4
4,Princess Of Wales Theatre,Theater,300 King St. W,CA,Toronto,Canada,at John St.,309,"[300 King St. W (at John St.), Toronto ON M5V ...","[{'label': 'display', 'lat': 43.64654315405365...",43.646543,-79.389100,NaN,M5V 1J2,ON,4ad4c061f964a520aef720e3
5,Le Germain Hotel Toronto Mercer,Hotel,30 Mercer St,CA,Toronto,Canada,NaN,142,"[30 Mercer St, Toronto ON M5V 1H3, Canada]","[{'label': 'display', 'lat': 43.64566900069133...",43.645669,-79.391044,NaN,M5V 1H3,ON,4ad4c05cf964a520baf520e3
6,barre3,Gym / Fitness Center,430 Adelaide St. West,CA,Toronto,Canada,NaN,213,"[430 Adelaide St. West, Toronto ON M5V 1S7, Ca...","[{'label': 'display', 'lat': 43.64631751622539...",43.646318,-79.390940,NaN,M5V 1S7,ON,54c3c353498eeda167779797
7,Akira Back,Japanese Restaurant,80 Blue Jays Way,CA,Toronto,Canada,NaN,105,"[80 Blue Jays Way, Toronto ON M5V 2G3, Canada]","[{'label': 'display', 'lat': 43.64537578930599...",43.645376,-79.392063,NaN,M5V 2G3,ON,5a2c772450a6f0229d83d923
8,General Assembly,Pizza Place,331 Adelaide Street West,CA,Toronto,Canada,NaN,336,"[331 Adelaide Street West, Toronto ON M5V, Can...","[{'label': 'display', 'lat': 43.64732611593410...",43.647326,-79.393043,Entertainment District,M5V,ON,5a219ab81c0b34450e104de7
9,Victor Restaurant & Bar,Bar,30 Mercer Street,CA,Toronto,Canada,at John St,136,"[30 Mercer Street (at John St), Toronto ON M5V...","[{'label': 'display', 'lat': 43.64563436248102...",43.645634,-79.391125,NaN,M5V 1H3,ON,4ad4c05df964a5203ff620e3


### Let's visualize the items around the Toronto Hilton City Centre 

In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map